![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `mixtral_8x7b_instruct_v01_q` to generate code based on instruction

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support for code generating in watsonx. It introduces commands for defining prompt and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to generate code using IBM `mixtral_8x7b_instruct_v01_q` watsonx model based on instruction provided by the user.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Generate code based on instruction](#generate)
- [Generated code testing](#testing)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/admin/create-services.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).


### Install dependecies

In [ ]:
!pip install "ibm-watsonx-ai>=0.1.7" | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [1]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#ibm_watsonx_ai.foundation_models.utils.enums.ModelTypes" target="_blank" rel="noopener no referrer">documentation</a>.

In [3]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT']


You need to specify `model_id` that will be used for inferencing:

In [4]:
model_id = "ibm-mistralai/mixtral-8x7b-instruct-v01-q"

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [5]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 200,
    GenParams.STOP_SEQUENCES: ["<end·of·code>"]
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [6]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

### Model's details

In [7]:
model.get_details()

{'model_id': 'ibm-mistralai/mixtral-8x7b-instruct-v01-q',
 'label': 'mixtral-8x7b-instruct-v01-q',
 'provider': 'IBM',
 'source': 'Hugging Face',
 'short_description': "Mixtral-8-7b-instruct-v01-gptq model is made with AutoGPTQ, which mainly leverages the quantization technique to 'compress' the model weights from FP16 to 4-bit INT and performs 'decompression' on-the-fly before computation (in FP16)",
 'long_description': "This model is made with AutoGPTQ, which mainly leverages the quantization technique to 'compress' the model weights from FP16 to 4-bit INT and performs 'decompression' on-the-fly before computation (in FP16). As a result, the GPU memory, and the data transferring between GPU memory and GPU compute engine, compared to the original FP16 model, is greatly reduced. The major quantization parameters used in the process are listed below.",
 'tier': 'class_2',
 'number_params': '7b',
 'min_shot_size': 1,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_au

<a id="generate"></a>
## Generate code based on instruction

Define instructions for the model with at-least one example.

In [8]:
instruction = """Using the directions below, generate Python code for the given task.

Input:
# Write a Python function that prints 'Hello World!' string 'n' times.

Output:
def print_n_times(n):
    for i in range(n):
        print("Hello World!")

<end of code>
"""

Prepare question for the model.

In [9]:
question = """Input:
# Write a Python function, which generates sequence of prime numbers.
# The function 'primes' will take the argument 'n', an int. It will return a list which contains all primes less than 'n'."""

### Generat the code using IBM `mixtral_8x7b_instruct_v01_q` model.


Inter the model to generate the code, according to provided instruction.

In [10]:
result = model.generate_text(" ".join([instruction, question]))

Formatting the text to get the function itself

In [11]:
code_as_text = result.split('Output:')[1].split('<end of code>')[0]

<a id="testing"></a>
## Generated code testing

The resulting code looks as below.

In [12]:
print(code_as_text)


def primes(n):
    primes = []
    for possiblePrime in range(2, n):
        isPrime = True
        for num in range(2, int(possiblePrime ** 0.5) + 1):
            if possiblePrime % num == 0:
                isPrime = False
                break
        if isPrime:
            primes.append(possiblePrime)
    return primes




Use generated code to make it as function.

In [13]:
exec(code_as_text)

Define the number 'n' for which the primes() function should process prime numbers.

In [18]:
n = 25

Test and run the generated function.

In [19]:
primes(n)

[2, 3, 5, 7, 11, 13, 17, 19, 23]

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to generate code based on instuction with IBM `mixtral_8x7b_instruct_v01_q` on watsonx. 
 
 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 

 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.